### jupyter notebook for the exploratory data analysis of the onet data

author: Jan Jörg
date: 14.03.2024

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Print the first 5 rows of the DataFrame
print(df.head())

# Print the summary statistics of the DataFrame
print(df.describe(include='all'))

# Print the information of the DataFrame
print(df.info())

# Plot histograms for numeric columns
df.select_dtypes(include=['int64', 'float64']).hist(figsize=(10, 10))

# For categorical columns, we can use countplot to see the distribution of categories
for column in df.select_dtypes(include=['object']).columns:
    plt.figure(figsize=(10, 4))
    sns.countplot(data=df, x=column)
    plt.xticks(rotation=90)
    plt.show()

NameError: name 'df' is not defined